# Land cover data

- Websites: [ESA-CCI-landcover](https://www.esa-landcover-cci.org/), [Viewer](https://maps.elie.ucl.ac.be/CCI/viewer/), [Publications](https://www.esa-landcover-cci.org/?q=node/184), [Maps legend](https://maps.elie.ucl.ac.be/CCI/viewer/download/CCI-LC_Maps_Legend.pdf), and [Quick user guide](https://maps.elie.ucl.ac.be/CCI/viewer/download/CCI-LC_Maps_Legend.pdf).
- This is a heavy data set. Just export preprocessed data and try to plot it later in the server!

## Initial setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Load packages.
import sys
import glob

import xarray as xr
import numpy as np

from dask.diagnostics import ProgressBar

In [4]:
print(">>> Version of xarray:", xr.__version__)

>>> Version of xarray: 0.14.0


In [5]:
# Map borders.
loni, lonf, lati, latf = -90, -30, 20, -60

In [6]:
# Folder with data.
DATA_FOLDER = "/media/alex/ALEXDATA/data_sets/ESA_CCI/LC/"

## Load data

In [7]:
# List files (just one).
FILE = "ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7b.nc"

# DataSet object.
DS = xr.open_dataset(
    DATA_FOLDER + FILE, 
    drop_variables=["processed_flag", 
                    "current_pixel_state",
                    "observation_count",
                    "change_count",
                    "crs"]
)

# South America.
DS = DS.sel(lon=slice(loni, lonf), lat=slice(lati, latf))
DA = DS.lccs_class

In [8]:
DS

<xarray.Dataset>
Dimensions:     (lat: 28800, lon: 21600)
Coordinates:
  * lat         (lat) float32 19.998611 19.995832 ... -59.995834 -59.99861
  * lon         (lon) float32 -89.99861 -89.995834 ... -30.004168 -30.001389
Data variables:
    lccs_class  (lat, lon) float32 ...
Attributes:
    title:                      ESA CCI Land Cover Map
    summary:                    This dataset contains the global ESA CCI land...
    type:                       ESACCI-LC-L4-LCCS-Map-300m-P1Y
    id:                         ESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7
    project:                    Climate Change Initiative - European Space Ag...
    references:                 http://www.esa-landcover-cci.org/
    institution:                Universite catholique de Louvain
    contact:                    landcover-cci@uclouvain.be
    comment:                    
    Conventions:                CF-1.6
    standard_name_vocabulary:   NetCDF Climate and Forecast (CF) Standard Nam...
    keywords

In [9]:
# Copy of attributes of this DataArray object.
DAattrs = DA.attrs
DA

<xarray.DataArray 'lccs_class' (lat: 28800, lon: 21600)>
[622080000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 19.998611 19.995832 ... -59.995834 -59.99861
  * lon      (lon) float32 -89.99861 -89.995834 ... -30.004168 -30.001389
Attributes:
    long_name:            Land cover class defined in LCCS
    standard_name:        land_cover_lccs
    flag_values:          [   0   10   11   12   20   30   40   50   60   61 ...
    flag_meanings:        no_data cropland_rainfed cropland_rainfed_herbaceou...
    valid_min:            1
    valid_max:            220
    ancillary_variables:  processed_flag current_pixel_state observation_coun...

In [10]:
# Load data into memory. Less space into memory because of int type ;)
DA = DA.astype(np.int16)

# Maintain original attributes.
DA = DA.assign_attrs(DAattrs)

In [11]:
print(">>> DataArray size in GB:", DA.nbytes / 1e9)
print(">>> DataSet size in GB:", DS.nbytes / 1e9)

>>> DataArray size in GB: 1.24416
>>> DataSet size in GB: 2.4885216


## Flags

In [12]:
# Make a dictionary with flag values / means.
flags_dict = {}
k = DA.flag_meanings.split(sep=" ")
v = DA.flag_values
for i in range(len(k)):
    flags_dict[k[i]] = v[i]
    
flags_dict    

{'no_data': 0,
 'cropland_rainfed': 10,
 'cropland_rainfed_herbaceous_cover': 11,
 'cropland_rainfed_tree_or_shrub_cover': 12,
 'cropland_irrigated': 20,
 'mosaic_cropland': 30,
 'mosaic_natural_vegetation': 40,
 'tree_broadleaved_evergreen_closed_to_open': 50,
 'tree_broadleaved_deciduous_closed_to_open': 60,
 'tree_broadleaved_deciduous_closed': 61,
 'tree_broadleaved_deciduous_open': 62,
 'tree_needleleaved_evergreen_closed_to_open': 70,
 'tree_needleleaved_evergreen_closed': 71,
 'tree_needleleaved_evergreen_open': 72,
 'tree_needleleaved_deciduous_closed_to_open': 80,
 'tree_needleleaved_deciduous_closed': 81,
 'tree_needleleaved_deciduous_open': 82,
 'tree_mixed': 90,
 'mosaic_tree_and_shrub': 100,
 'mosaic_herbaceous': 110,
 'shrubland': 120,
 'shrubland_evergreen': 121,
 'shrubland_deciduous': 122,
 'grassland': -126,
 'lichens_and_mosses': -116,
 'sparse_vegetation': -106,
 'sparse_tree': -105,
 'sparse_shrub': -104,
 'sparse_herbaceous': -103,
 'tree_cover_flooded_fresh_or_br

**Manual correction for flags dict!** I am not `completely` sure about this! 

In [13]:
keys = [
    "grassland", "lichens_and_mosses", 
    "sparse_vegetation", "sparse_tree", 
    "sparse_shrub", "sparse_herbaceous", 
    "tree_cover_flooded_fresh_or_brakish_water", "tree_cover_flooded_saline_water", 
    "shrub_or_herbaceous_cover_flooded", "urban", 
    "bare_areas", "bare_areas_consolidated", 
    "bare_areas_unconsolidated", "water", 
    "snow_and_ice"
]

values = [
    130, 140,
    150, 151,
    152, 153,
    160, 170, 
    180, 190,
    200, 201,
    202, 210, 
    220
]

for key, value in zip(keys, values):
    flags_dict[key] = value

# Corrected dictionary.
flags_dict

{'no_data': 0,
 'cropland_rainfed': 10,
 'cropland_rainfed_herbaceous_cover': 11,
 'cropland_rainfed_tree_or_shrub_cover': 12,
 'cropland_irrigated': 20,
 'mosaic_cropland': 30,
 'mosaic_natural_vegetation': 40,
 'tree_broadleaved_evergreen_closed_to_open': 50,
 'tree_broadleaved_deciduous_closed_to_open': 60,
 'tree_broadleaved_deciduous_closed': 61,
 'tree_broadleaved_deciduous_open': 62,
 'tree_needleleaved_evergreen_closed_to_open': 70,
 'tree_needleleaved_evergreen_closed': 71,
 'tree_needleleaved_evergreen_open': 72,
 'tree_needleleaved_deciduous_closed_to_open': 80,
 'tree_needleleaved_deciduous_closed': 81,
 'tree_needleleaved_deciduous_open': 82,
 'tree_mixed': 90,
 'mosaic_tree_and_shrub': 100,
 'mosaic_herbaceous': 110,
 'shrubland': 120,
 'shrubland_evergreen': 121,
 'shrubland_deciduous': 122,
 'grassland': 130,
 'lichens_and_mosses': 140,
 'sparse_vegetation': 150,
 'sparse_tree': 151,
 'sparse_shrub': 152,
 'sparse_herbaceous': 153,
 'tree_cover_flooded_fresh_or_brakish_

In [14]:
# Correction of attributes.
DA.attrs["flag_values"] = np.array([])
DA.attrs["flag_meanings"] = np.array([])

for k, v in flags_dict.items():
    DA.attrs["flag_values"] = np.append(DA.attrs["flag_values"], v)
    DA.attrs["flag_meanings"] = np.append(DA.attrs["flag_meanings"], k)
    
# As int.    
DA.attrs["flag_values"] = DA.flag_values.astype(int)    

In [15]:
DA.attrs

{'long_name': 'Land cover class defined in LCCS',
 'standard_name': 'land_cover_lccs',
 'flag_values': array([  0,  10,  11,  12,  20,  30,  40,  50,  60,  61,  62,  70,  71,
         72,  80,  81,  82,  90, 100, 110, 120, 121, 122, 130, 140, 150,
        151, 152, 153, 160, 170, 180, 190, 200, 201, 202, 210, 220]),
 'flag_meanings': array(['no_data', 'cropland_rainfed', 'cropland_rainfed_herbaceous_cover',
        'cropland_rainfed_tree_or_shrub_cover', 'cropland_irrigated',
        'mosaic_cropland', 'mosaic_natural_vegetation',
        'tree_broadleaved_evergreen_closed_to_open',
        'tree_broadleaved_deciduous_closed_to_open',
        'tree_broadleaved_deciduous_closed',
        'tree_broadleaved_deciduous_open',
        'tree_needleleaved_evergreen_closed_to_open',
        'tree_needleleaved_evergreen_closed',
        'tree_needleleaved_evergreen_open',
        'tree_needleleaved_deciduous_closed_to_open',
        'tree_needleleaved_deciduous_closed',
        'tree_needleleave

In [16]:
# Export data as a netcdf file.
DA.to_netcdf(DATA_FOLDER + "ppdata_2015_south_america.nc")